In [1]:
import configparser
import psycopg2
from sql_queries import create_table_queries, drop_table_queries


def drop_tables(cur, conn):
    for query in drop_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()


def main():
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()

    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()

DROP TABLE IF EXISTS staging_events
DROP TABLE IF EXISTS staging_songs
DROP TABLE IF EXISTS songplay
DROP TABLE IF EXISTS users
DROP TABLE IF EXISTS songs
DROP TABLE IF EXISTS artists
DROP TABLE IF EXISTS time


In [2]:
import configparser
import psycopg2
from sql_queries import copy_table_queries, insert_table_queries


def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

def insert_tables(cur, conn):
    for query in insert_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

def main():
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
    
    load_staging_tables(cur, conn)
    insert_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()


    copy staging_events from 's3://udacity-dend/log_data'
    credentials 'aws_iam_role=arn:aws:iam::110255064301:role/myRedshiftRole'
    region 'us-west-2' 
    compupdate off 
    format as JSON 's3://udacity-dend/log_json_path.json'


    copy staging_songs from 's3://udacity-dend/song_data'
    credentials 'aws_iam_role=arn:aws:iam::110255064301:role/myRedshiftRole'
    region 'us-west-2' 
    compupdate off
    JSON 'auto'
    

    INSERT INTO songplays (start_time, user_id, level, song_id, 
                            artist_id, session_id, location, user_agent) 
    SELECT DISTINCT 
        events.ts, 
        events.user_id, 
        events.user_level,
        songs.song_id,
        songs.artist_id,
        events.session_id,
        events.location,
        events.user_agent
    FROM staging_events as events, staging_songs as songs
    WHERE events.page = 'NextSong'
    AND events.title = songs.title
    AND user_id NOT IN (SELECT DISTINCT songs.user_id FROM songplays as so